In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 129kB/s 
     |████████████████████████████████| 983kB 42.6MB/s 
     |████████████████████████████████| 245kB 44.0MB/s 
     |████████████████████████████████| 1.1MB 45.3MB/s 
     |████████████████████████████████| 471kB 46.1MB/s 
     |████████████████████████████████| 890kB 42.0MB/s 
     |████████████████████████████████| 3.0MB 15.7MB/s 
     |████████████████████████████████| 1.1MB 50.0MB/s 
  Created wheel for ktrain: filename=ktrain-0.23.0-cp36-none-any.whl size=25271671 sha256=f6249f4d87e499df835a2e4e28152391efdba035eb4e43e1a0f1279eaa6bfce6
  Stored in directory: /root/.cache/pip/wheels/f2/15/d1/ada9c041daebeb09b7341e2d3d0a765bf74e512edf2caa6fa7
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=32b3e2131efe4ef70ae23a6a9217a7d5cb6cd875fd63295382ae1b98a045eb24
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

# New Section

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import ktrain
from ktrain import text

In [ ]:
### read the full set of narrtives set as either Accident or Incident
df = pd.read_pickle("/content/FULLSET")

In [ ]:
### split the dataset randomly into two files
### one for trianign and one for testing

data_train=df.sample(frac=0.8,random_state=200) #random state is a seed value
data_test=df.drop(data_train.index)

In [ ]:
data_train.head()

,Type,Narrtive
15811,Accident,THE A/C LANDED AND TAXIED TO THE RAMP. BEFORE...
21416,Accident,"AFTER A SHORT ROLLING T/O, THE PILOT NOTICED ..."
8393,Incident,PRN: ON A RETURN TRIP TO CAMP WITH AN EMPTY N...
9995,Incident,THE MGB (MAIN GEARBOX) CHIP DETECTOR ILLUMINA...
22427,Accident,During agricultural operation over a soya pla...


In [ ]:
(X_train, y_train),(X_test, y_test),preprocess=text.texts_from_df(train_df=data_train,
                  text_column='Narrtive',
                  label_columns = 'Type',
                  val_df = data_test,
                  maxlen=400,
                  preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
X_train[0].shape

(30650, 400)

In [ ]:
model = text.text_classifier(name='bert',
                            train_data=(X_train, y_train),
                            preproc=preprocess)

Is Multi-Label? False
maxlen is 400
done.


In [ ]:
# Get learignig rate
learner = ktrain.get_learner(model = model,
                            train_data=(X_train, y_train),
                            val_data=(X_test, y_test),
                            batch_size=6)

In [ ]:
# this might take days to run
# learner.lr_find()
# learner.lr_plot()
# optimal learnign rate for thie model is 2e-5

In [ ]:
learner.fit_onecycle(lr = 2.5e-5, epochs=1)



begin training using onecycle policy with max lr of 2.5e-05...
5109/5109 [==============================] - 4395s 860ms/step - loss: 0.2673 - accuracy: 0.8905 - val_loss: 0.2112 - val_accuracy: 0.9109


In [ ]:
predictor = ktrain.get_predictor(learner.model, preprocess)

In [ ]:
data = ['Airplane took off and was lost',
        'The plane hit some turbulence, no one was injured.  and there was minor damage',
        'The aircraft experiences minor damange']

In [ ]:
predictor.predict(data)

['Accident', 'Incident', 'Incident']

In [ ]:
predictor.save('/content/ADREPBERT')

In [ ]:
!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

  Cloning https://github.com/amaiya/eli5 (to revision tfkeras_0_10_1) to /tmp/pip-req-build-kp6owpqs
  Running command git clone -q https://github.com/amaiya/eli5 /tmp/pip-req-build-kp6owpqs
  Running command git checkout -b tfkeras_0_10_1 --track origin/tfkeras_0_10_1
  Switched to a new branch 'tfkeras_0_10_1'
  Branch 'tfkeras_0_10_1' set up to track remote branch 'tfkeras_0_10_1' from 'origin'.
  Created wheel for eli5: filename=eli5-0.10.1-py2.py3-none-any.whl size=106831 sha256=bcdfae0aa88c4752c4cced84087d54e0c434b3b739af64c07da29c3f12c0fbe9
  Stored in directory: /tmp/pip-ephem-wheel-cache-dycpx00l/wheels/51/59/0a/0f48442b8d209583a4453580938d7ba2270aca40edacee6d45
Successfully built eli5


In [ ]:
predictor.explain('The plane hit some turbulence, no one was injured')

save the model json file

In [ ]:
model_json = learner.model.to_json()

In [ ]:
with open("/content/model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
learner.model.save_weights("/content/model.h5")

In [ ]:
loaded_model = ktrain.load_predictor('/content/ADREPBERT')

In [ ]:
loaded_model.predict(data)

['Accident', 'Incident', 'Incident']

# New Section